In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf 
from tensorflow.keras.layers import Dense
import tensorflow_hub as hub

In [5]:
train = pd.read_csv('../input/nlp-getting-started/train.csv')
test = pd.read_csv('../input/nlp-getting-started/test.csv')
submission = pd.read_csv('../input/nlp-getting-started/sample_submission.csv')
tweets = train['text']
targets = train['target']
use_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], 
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE",
                          )
train_text , val_text , train_label, val_label = train_test_split(train['text'].to_numpy(),
                                                                          train['target'].to_numpy(),
                                                                          test_size=0.1,random_state=4)


In [14]:
test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [6]:
model = tf.keras.Sequential([
    use_layer,
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(16),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE (KerasLayer)             (None, 512)               256797824 
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [7]:
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy']) 

In [9]:
model.fit(train_text,train_label, epochs = 100, validation_data = (val_text,val_label))

Epoch 1/100
215/215 [==============================] - 9s 20ms/step - loss: 0.5717 - accuracy: 0.7320 - val_loss: 0.5790 - val_accuracy: 0.7310
Epoch 2/100
215/215 [==============================] - 3s 15ms/step - loss: 0.4499 - accuracy: 0.8066 - val_loss: 0.4734 - val_accuracy: 0.8045
Epoch 3/100
215/215 [==============================] - 3s 14ms/step - loss: 0.4071 - accuracy: 0.8269 - val_loss: 0.4280 - val_accuracy: 0.8136
Epoch 4/100
215/215 [==============================] - 3s 16ms/step - loss: 0.3851 - accuracy: 0.8337 - val_loss: 0.4313 - val_accuracy: 0.8241
Epoch 5/100
215/215 [==============================] - 3s 15ms/step - loss: 0.3595 - accuracy: 0.8533 - val_loss: 0.4364 - val_accuracy: 0.8281
Epoch 6/100
215/215 [==============================] - 3s 14ms/step - loss: 0.3497 - accuracy: 0.8542 - val_loss: 0.4581 - val_accuracy: 0.8215
Epoch 7/100
215/215 [==============================] - 3s 14ms/step - loss: 0.3332 - accuracy: 0.8562 - val_loss: 0.4584 - val_accuracy:

In [15]:
prediction = model.predict(test['text'])

In [16]:
prediction
for i in range(len(prediction)):
    if(prediction[i]>0.5):
        prediction[i]=1
    else:
        prediction[i]=0

In [20]:
prediction= prediction.astype(int)

In [21]:
submission['target'] = prediction
submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [22]:

submission.to_csv('predictions4.csv', header=False, index=False)

In [37]:
use_layer(test_data)

<tf.Tensor: shape=(3263, 512), dtype=float32, numpy=
array([[-0.06240411,  0.06200171, -0.01510903, ..., -0.06985617,
        -0.02305336,  0.05401625],
       [-0.01334469,  0.01704533, -0.05602343, ...,  0.01448949,
        -0.04619199, -0.05617152],
       [ 0.05268431, -0.01391451,  0.04169051, ...,  0.04957249,
        -0.00793166,  0.0492492 ],
       ...,
       [ 0.07666268,  0.00141271,  0.03278234, ...,  0.00271717,
        -0.03690447, -0.04324871],
       [ 0.07673816,  0.00217168, -0.06574667, ...,  0.01257792,
         0.0316073 ,  0.02010439],
       [ 0.04707664, -0.00459063, -0.02711707, ..., -0.00465905,
        -0.00946341,  0.02349724]], dtype=float32)>

In [ ]:
pred = model.predict(test['text'])
pred = pred.round().astype('int32')
submission['target'] = pred
submission.to_csv('submission.csv',index=False)